In [ ]:
import paho.mqtt.client as mqtt
import mysql.connector
from datetime import datetime

# Inisialisasi variabel global
node_id = 0
soil1_value = 0
soil2_value = 0
rain_value = False
battery_value = False
acceleration_x = 0
acceleration_y = 0
acceleration_z = 0

# Fungsi yang dipanggil saat mendapatkan pesan MQTT
def on_message(client, userdata, msg):
    global node_id, soil1_value, soil2_value, rain_value, battery_value, acceleration_x, acceleration_y, acceleration_z

    topic = msg.topic
    payload = msg.payload.decode("utf-8")

    # Parsing topik dan memasukkan nilai ke variabel sesuai dengan topik
    if topic == "/sensor/node_id":
        node_id = int(payload)
    elif topic == "/sensor/soil1":
        soil1_value = int(payload)
    elif topic == "/sensor/soil2":
        soil2_value = int(payload)
    elif topic == "/sensor/rain":
        rain_value = bool(int(payload))
    elif topic == "/sensor/battery":
        battery_value = bool(int(payload))
    elif topic == "/sensor/acceleration_x":
        acceleration_x = int(payload)
    elif topic == "/sensor/acceleration_y":
        acceleration_y = int(payload)
    elif topic == "/sensor/acceleration_z":
        acceleration_z = int(payload)

    # Insert data ke dalam database MySQL
    insert_data(node_id, soil1_value, soil2_value, rain_value, battery_value, acceleration_x, acceleration_y, acceleration_z)

# Fungsi untuk menyimpan data ke dalam database MySQL
def insert_data(node_id, soil1_value, soil2_value, rain_value, battery_value, acceleration_x, acceleration_y, acceleration_z):
    conn = mysql.connector.connect(
        host="localhost",
        user="root",
        password="",
        database="IOT"
    )
    cursor = conn.cursor()

    current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

       # Insert data ke dalam tabel NodeSensor
    cursor.execute("""
        INSERT INTO NodeSensor
        (node_id, kondisi_battery, status_hujan,
         data_kelembapan_tanah_1, data_kelembapan_tanah_2,
         data_kemiringan_x, data_kemiringan_y, data_kemiringan_z,
         data_lokasi, timestamp)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, (node_id, battery_value, rain_value, soil1_value, soil2_value, acceleration_x, acceleration_y, acceleration_z, None, current_time))


    conn.commit()
    conn.close()

# Inisialisasi MQTT client
client = mqtt.Client()
client.on_message = on_message

# Isi sesuai dengan broker MQTT yang Anda gunakan
mqtt_broker_address = "0.tcp.ap.ngrok.io"
mqtt_broker_port = 18777

# Isi sesuai dengan topik yang digunakan pada Arduino
mqtt_topic = "/sensor/#"

# Connect ke broker MQTT
client.connect(mqtt_broker_address, mqtt_broker_port, 60)

# Subscribe ke topik yang diinginkan
client.subscribe(mqtt_topic)

# Loop forever untuk mendengarkan pesan
client.loop_forever()
